# N-panel plots of IRIS data and velocity dealiasing
_______________________________________________________________________________________________

This script takes an IRIS raw volume PPI scan file as input and generates a N-panel plot for each sweep in the volume  file. Each of the output plots consists of a row of panels with a panel for each selected data type. The script also uses PyArt dealiasing algorithms (phase unwrap and region based) to correct the Doppler velocity data and add corresponding corrected velocity data fields to the radar object.

Here an example with a failure of the unwrap phase algorithm. Region based works nicely though!

![description](files/figs/CDV130618145623_el6_Vdealias.png)


In [ ]:
#! /usr/bin/env python

import matplotlib.pyplot as plt
import pylab as plb
import matplotlib as mpl
import pyart


In [ ]:
## SETTINGS #####################################################################

in_path = './data/'
out_path = './output/'
filename = 'CDV130618145623.RAWCBRF'
radar_abbr = filename[:3]

lims = [150, 125, 100, 50, 50, 40, 30, 20]

sel_data = ['reflectivity', 'velocity', 'corrected_velocity_uwp', 'corrected_velocity_reg']

In [ ]:
Ndata = len(sel_data)

In [ ]:
## COLORMAP #####################################################################

cdict = {'red': ((0.0, 0.0, 0.0),
                     (0.25, 0.0, 0.0),
                     (0.5, 0.9, 0.9),
                     (0.75, 1.0, 1.0),
                     (1.0, 1.0, 1.0)),
         'green': ((0.0, 0.5, 0.5),
                   (0.25, 0., 0.),
                   (0.5, 0.9, 0.9),
                   (0.75, 0., 0.),
                   (1.0, 0.5, 0.5)),
         'blue': ((0.0, 1.0, 1.0),
                  (0.25, 1., 1.),
                  (0.5, 0.9, 0.9),
                  (0.75, 0.0, 0.0), 
                  (1.0, 0.0, 0.0))}

                  
my_cmap=mpl.colors.LinearSegmentedColormap('my_colormap', cdict, N=33)

In [ ]:
## DATA ##########################################################################

in_file = in_path + filename

radar = pyart.io.read_rsl(in_file)
radar.metadata['instrument_name'] = radar_abbr

nyq_vel = radar.instrument_parameters['nyquist_velocity']['data'][0]
sw_num = radar.nsweeps
sw_elevs = [radar.fixed_angle['data'][sw] for sw in range(0, sw_num-1)]

dealias_data_uwp = pyart.correct.dealias_unwrap_phase(radar, rays_wrap_around=True, keep_original=False)
radar.add_field('corrected_velocity_uwp', dealias_data_uwp)

dealias_data_reg = pyart.correct.dealias_region_based(radar, interval_splits=20, rays_wrap_around=True, keep_original=False)
radar.add_field('corrected_velocity_reg', dealias_data_reg)

In [21]:
## PLOTS ########################################################################

display = pyart.graph.RadarDisplay(radar)
time_text = ' ' + display.time_begin.isoformat() + 'Z '

st_range = -150
end_range = 150
i_range = -150
f_range = 150

for sw in range(0, sw_num-1):

    if f_range>50:
        i_range = st_range + sw*25
        f_range = end_range - sw*25
    else:
        i_range = -30
        f_range = 30

    out_file = out_path + filename.split('.', 1)[0]+ '_el%.0f' % (sw_elevs[sw]) + '_Vdealias.png'
    fig = plt.figure(figsize=(36,6.5))
    
    pl_count = 1
    for dd in sel_data:
        
        data_name = radar.fields[dd]['long_name']
        data_units = radar.fields[dd]['units']
        
        cb_lab = data_name + '\n' + data_units
        title = radar_abbr + time_text + 'elev %.1f' % (sw_elevs[sw]) + 'deg' + '\n' + dd 
        
        ax = fig.add_subplot(100+Ndata*10+pl_count)
        
        display.plot(dd, sw, vmin=-2*nyq_vel, vmax=2*nyq_vel, 
        colorbar_label=cb_lab, 
        cmap=my_cmap, title=title, ax=ax, mask_outside=False)
        
        display.plot_range_rings(range(25, 150, 25))
        display.plot_cross_hair(0.5)
        plt.xlim((-lims[sw], lims[sw]))
        plt.ylim((-lims[sw], lims[sw]))

        plt.savefig(out_file)
        
        pl_count += 1
    
    plt.close()
